In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from src.utils import *

In [3]:
os.chdir('..')

In [4]:
import yaml

with open(get_config_path("data_config.yaml"), "r") as f:
    config = yaml.safe_load(f)

In [5]:
def seconds_to_frames(seconds, sample_rate=config["sample_rate"], hop_length=config["hop_length"]):
    frames = int(seconds * sample_rate / hop_length)
    return frames

In [6]:
# symbols = {}
# for filepath in os.listdir(get_alignment_dir()):
#     if not filepath.endswith(".TextGrid"):
#         continue
#     alignment = get_alignment_path(filepath)
#     with open(alignment, "r") as f:
#         lines = f.readlines()
#         
#     found_phones = False
#     for i, line in enumerate(lines):
#         if 'item [2]:' in line:
#             found_phones = True
#         if found_phones and 'text =' in line:
#             phone = line.split('=')[1].strip().strip('"')
#             if phone not in symbols:
#                 symbols[phone] = len(symbols)
#                 
# symbols["<pad>"] = len(symbols)

In [7]:
from scripts.prepare_dataset import get_mel_spectrogram

example_wav = get_audio_path("LJ001-0001.wav")
mel_spectrogram = get_mel_spectrogram(example_wav, config)
mel_spectrogram.shape

LibsndfileError: Error opening '/Users/martinledl/Developer/nano-tts/data/LJSpeech-1.1/wavs/LJ001-0001.wav': System error.

/Users/martinledl/Developer/ml-venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py:211: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if not hasattr(module, "__file__") or module.__file__ is None:


In [9]:
import torchaudio
from speechbrain.inference.vocoders import HIFIGAN
from speechbrain.lobes.models.FastSpeech2 import mel_spectogram

# Load a pretrained HIFIGAN Vocoder
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="pretrained_models/tts-hifigan-ljspeech")

# Compute the mel spectrogram.
# IMPORTANT: Use these specific parameters to match the Vocoder's training settings for optimal results.
spectrogram = get_mel_spectrogram(example_wav, config)

# Convert the spectrogram to waveform
waveforms = hifi_gan.decode_batch(spectrogram)

# Save the reconstructed audio as a waveform
# torchaudio.save('waveform_reconstructed.wav', waveforms.squeeze(1), 22050)


/Users/martinledl/Developer/ml-venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [9]:
import torch

data = torch.load('data/processed/LJ001-0001.pt')
data['phonemes'].shape, data['durations'].shape, data['mel_spectrogram'].shape

(torch.Size([114]), torch.Size([114]), torch.Size([80, 832]))